In [ ]:
#TODO filter years so it'll match the years in the other dataset (awards mostly)
#TODO manage the missing values and NaNs
#TODO manage the outliers


In [1]:
import pandas as pd

#open the csv file of actors
actors = pd.read_csv('actors.csv')

#open the csv file of awards
awards = pd.read_csv('data_csv_awards.csv')

#open the movie tsv file
movies = pd.read_csv('movie_summaries_ada/movie.metadata.tsv', sep='\t', header=None)
#define the columns
movies.columns = ['wikipedia_id', 'freebase_id', 'name', 'release_date', 'revenue', 'runtime', 'languages', 'countries', 'genres']

#open the character.metadata.tsv file
characters = pd.read_csv('movie_summaries_ada/character.metadata.tsv', sep='\t', header=None)
#define the columns of the character file
characters.columns = ['wikipedia_movie_id','freebase_movie_id','movie_release_date','character_name','actor_birth',
                      'actor_gender','actor_height','actor_etnicity','actor_name','actor_age_at_release','freebase_char_actor_map_id','freebase_character_id','freebase_actor_id']

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
#load the heavy files aka titles imbd to find the imdb unique id, then use it to find the imdb rating and merge it with the movies dataframe
titles_imbd = pd.read_csv('IMDB/title.akas.tsv', sep='\t')

# where only keep the region as US or GB, only the ones that could have an impact in the US

#filter the titles_imbd dataframe to only keep the rows with the region as US or GB
titles_imbd = titles_imbd[(titles_imbd.region == 'US') | (titles_imbd.region == 'GB')]
#filter the titles_imbd dataframe if same titleId only the first one is kept
titles_imbd = titles_imbd.drop_duplicates(subset=['titleId'], keep='first')
#only keep the columns that we need
titles_imbd = titles_imbd[['titleId', 'title', 'region']]

print(titles_imbd.head())

/var/folders/nb/6_kh75dx3xb3cgzlclgmvcx00000gn/T/ipykernel_3907/146347062.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  titles_imbd = pd.read_csv('IMDB/title.akas.tsv', sep='\t')


In [4]:
#add the titleId column to the movies dataframe if the title is the same
movies = movies.merge(titles_imbd, left_on='name', right_on='title', how='left')
#drop the title and region columns
movies = movies.drop(columns=['title', 'region'])

,wikipedia_id,freebase_id,name,release_date,revenue,runtime,languages,countries,genres,titleId
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",NaN
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",NaN


In [9]:
#count the number of movies that have a titleId
print(str(movies.titleId.count())+' movies have a titleId over '+str(movies.name.count())+' movies, so '+str(round(movies.titleId.count()*100/movies.name.count()))+' % movies do have a titleId')

330051 movies have a titleId over 361410 movies, so 91 % movies do have a titleId


In [10]:
#load the imdb rating file
rating_imbd = pd.read_csv('IMDB/title.ratings.tsv', sep='\t')

In [ ]:
#add the rating to the movies dataframe
movies = movies.merge(rating_imbd, left_on='titleId', right_on='tconst', how='left')
#drop the tconst column
movies = movies.drop(columns=['tconst'])

In [16]:
print(str(movies.averageRating.count())+' movies have a rating over '+str(movies.titleId.count())+' movies, so '+str(round(movies.averageRating.count()*100/movies.titleId.count()))+' % movies do have a rating')

162237 movies have a rating over 330051 movies, so 49 % movies do have a rating


In [21]:
#filter the awards dataframe to only keep the rows with winner as True
awards_winners = awards[awards.winner == True]

#filter the awards dataframe to only keep the rows with category containing ACTOR or ACTRESS
awards_actors = awards_winners[awards_winners.category.str.contains('ACTOR|ACTRESS')]

In [ ]:
#count the number of awards per entity in awards_actors
awards_actors_count = awards_actors.groupby('entity',as_index=False).count()

In [58]:
# get a list of unique actors from the character dataframe
actors_list = pd.DataFrame(characters.actor_name.unique())
print(actors_list.shape)

#rename the column
actors_list.columns = ['name']

#add a column to the actors_list dataframe with the winners count
actors_list = actors_list.merge(awards_actors_count[['entity','winner']], left_on='name', right_on='entity', how='left')
#drop the entity column
actors_list = actors_list.drop(columns=['entity'])
#rename the count column
actors_list = actors_list.rename(columns={'winner': 'awards'})
#fill the NaN values in the awards column with 0
actors_list['awards'] = actors_list['awards'].fillna(0)
print(actors_list.shape)

#create a dataframe with the number of movies per actor
actors_movies_count = pd.DataFrame(characters.actor_name.value_counts())

#add a column to the actors_list dataframe with the movies count
actors_list = actors_list.merge(actors_movies_count, left_on='name', right_index=True, how='left')
#rename the count column
actors_list = actors_list.rename(columns={'actor_name': 'movies'})
print(actors_list.shape)

#replace the Actor column of actors dataframe by the same value but replace each _ by a space
actors['Actor'] = actors['Actor'].str.replace('_', ' ')

#add the actors dataframe to the actors_list dataframe on the Actor column and name column respectively
actors_list = actors_list.merge(actors, left_on='name', right_on='Actor', how='left')

#drop the Actor column
actors_list = actors_list.drop(columns=['Actor'])
print(actors_list.shape)
#print the number of actors with a Fame value
print(str(actors_list.Fame.count())+' actors have a Fame value over '+str(actors_list.name.count())+' actors, so '+str(round(actors_list.Fame.count()*100/actors_list.name.count()))+' % actors do have a Fame value')

#TODO:issue to solve, we loose about 46 actors in the merge because the name has . or ' or -, and we replace every _ by a space
#TODO: possible solution, replace every . or ' or - by a space in the actors_list dataframe
#TODO: possible solution, replace every . or ' or - or space by a _ in the actors dataframe

#TODO: NaN values to manage

(134079, 1)
(134079, 2)
(134079, 3)
(134079, 7)
1310 actors have a Fame value over 134078 actors, so 1 % actors do have a Fame value


In [59]:
print(actors_list.shape)
actors_list.head()

(134079, 7)


,name,awards,movies,Fame,Liked,Disliked,Neutral
0,Wanda De Jesus,0.0,8.0,NaN,NaN,NaN,NaN
1,Natasha Henstridge,0.0,23.0,NaN,NaN,NaN,NaN
2,Ice Cube,0.0,33.0,93.0,57.0,11.0,25.0
3,Jason Statham,0.0,31.0,78.0,59.0,4.0,14.0
4,Clea DuVall,0.0,31.0,NaN,NaN,NaN,NaN


In [61]:
print(movies.shape)
movies.head()

(361410, 12)


,wikipedia_id,freebase_id,name,release_date,revenue,runtime,languages,countries,genres,titleId,averageRating,numVotes
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",NaN,NaN,NaN
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",NaN,NaN,NaN
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",NaN,NaN,NaN
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",NaN,NaN,NaN
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",tt0083949,5.9,623.0


In [9]:
# basic info of the data to see changes when cleaning
print(actors.info())
print(awards.info())
print(movies.info())
print(characters.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1355 entries, 0 to 1354
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Actor     1355 non-null   object
 1   Fame      1355 non-null   int64 
 2   Liked     1355 non-null   int64 
 3   Disliked  1355 non-null   int64 
 4   Neutral   1355 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 53.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11058 entries, 0 to 11057
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   year      11058 non-null  int64 
 1   category  11058 non-null  object
 2   winner    11058 non-null  bool  
 3   entity    11058 non-null  object
dtypes: bool(1), int64(1), object(2)
memory usage: 270.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81741 entries, 0 to 81740
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        